# **Proje Hazırlığı ve Veriyi Anlama**

## Dosya Yapısı İncelemesi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import json
import os
from pathlib import Path

# Dosya yolunu ayarla
base_path = '/content/drive/MyDrive/Milano Telecom Data Analysis — 2013 Week Dataset/archive/'

## Trafik Verisi İncelemesi

In [ ]:
# 1. Trafik verisi örneği (ilk 100 satır)
print("=== TRAFİK VERİSİ İNCELEMESİ ===")
traffic_file = base_path + 'sms-call-internet-mi-2013-11-01.csv'

# Sütunları gör (çok büyük dosya, sadece header oku)
try:
    df_traffic_sample = pd.read_csv(traffic_file, nrows=100)
    print("Örnek veri (100 satır):")
    print(df_traffic_sample.head())
    print("\nSütunlar:", df_traffic_sample.columns.tolist())
    print("Boyut:", df_traffic_sample.shape)

    # Veri tipleri
    print("\nVeri tipleri:")
    print(df_traffic_sample.dtypes)

    # Null değerler
    print("\nNull değer özeti:")
    print(df_traffic_sample.isnull().sum())

except Exception as e:
    print(f"Hata: {e}")
    # Alternatif: sadece ilk satırı oku
    with open(traffic_file, 'r') as f:
        header = f.readline()
    print("Header:", header)

## Hareket Verisi İncelemesi

In [ ]:
print("\n=== HAREKET VERİSİ İNCELEMESİ ===")
movement_file = base_path + 'mi-to-provinces-2013-11-01.csv'

try:
    df_movement_sample = pd.read_csv(movement_file, nrows=100)
    print("Örnek veri (100 satır):")
    print(df_movement_sample.head())
    print("\nSütunlar:", df_movement_sample.columns.tolist())
    print("Boyut:", df_movement_sample.shape)

except Exception as e:
    print(f"Hata: {e}")
    with open(movement_file, 'r') as f:
        header = f.readline()
    print("Header:", header)

## Census Verisi İncelemesi

In [ ]:
print("\n=== CENSUS VERİSİ İNCELEMESİ ===")
census_file = base_path + 'ISTAT_census_variables_2011.csv'

df_census = pd.read_csv(census_file)
print("İlk 5 satır:")
print(df_census.head())
print("\nSütunlar:", df_census.columns.tolist())
print("Boyut:", df_census.shape)
print("\nVeri tipleri:")
print(df_census.dtypes)

## Geojson Dosyalarının İncelenmesi

In [ ]:
print("\n=== GEOJSON DOSYALARI ===")
# Milano grid
with open(base_path + 'milano-grid.geojson', 'r') as f:
    milano_grid = json.load(f)
print(f"Milano grid: {len(milano_grid['features'])} özellik")

# İtalya eyaletleri
with open(base_path + 'Italian_provinces.geojson', 'r') as f:
    italian_provinces = json.load(f)
print(f"İtalyan eyaletleri: {len(italian_provinces['features'])} özellik")

## Tüm Dosya Boyutlarının Kontrolü

In [ ]:
print("\n=== TÜM DOSYA LİSTESİ VE BOYUTLARI ===")
import os

# Tüm CSV ve JSON dosyalarını bul
all_files = []
for file in os.listdir(base_path):
    if file.endswith(('.csv', '.json', '.geojson')):
        all_files.append(file)

# Alfabetik sırala
all_files.sort()

print(f"Toplam {len(all_files)} dosya bulundu:")
print("-" * 50)

for i, file in enumerate(all_files, 1):
    file_path = os.path.join(base_path, file)
    size_mb = os.path.getsize(file_path) / (1024 * 1024)
    print(f"{i:2d}. {file:45s} - {size_mb:8.2f} MB")

print("-" * 50)
total_size_mb = sum(os.path.getsize(os.path.join(base_path, f)) / (1024 * 1024) for f in all_files)
print(f"Toplam boyut: {total_size_mb:.2f} MB ({total_size_mb/1024:.2f} GB)")

# **BigQuery'ye Dosya Yükleme Planı**

## Dosya Yolu ve Proje Bilgileri

In [ ]:
# Colab'da çalıştır
from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery
import pandas as pd
import geopandas as gpd
import json
from google.cloud.exceptions import NotFound

# Konfigürasyon
PROJECT_ID = "YOUR_PROJECT_ID"
DATASET_ID = "milano_mobile_2013"
BASE_PATH = "/content/drive/MyDrive/Milano Telecom Data Analysis — 2013 Week Dataset/archive/"

client = bigquery.Client(project=PROJECT_ID)

## Dataset Varlık Kontrolü

In [ ]:
# Dataset var mı kontrol et
dataset_ref = client.dataset(DATASET_ID)
try:
    client.get_dataset(dataset_ref)
    print(f"✅ Dataset bulundu: {DATASET_ID}")
except NotFound:
    print(f"❌ Dataset bulunamadı: {DATASET_ID}")
    # Eğer yoksa oluştur (manuel oluşturmuştunuz, bu gerekmez)

## Tabloların Yüklenmesi

In [ ]:
def load_all_tables():
    """Tüm dosyaları BigQuery'e yükler"""

    # 1. CENSUS VERİSİ
    print("📊 1. Census verisi yükleniyor...")
    census_path = BASE_PATH + "ISTAT_census_variables_2011.csv"
    df_census = pd.read_csv(census_path)

    table_id = f"{PROJECT_ID}.{DATASET_ID}.census_data"
    job_config = bigquery.LoadJobConfig(
        autodetect=True,
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE
    )

    job = client.load_table_from_dataframe(df_census, table_id, job_config=job_config)
    job.result()
    print(f"   ✅ census_data: {client.get_table(table_id).num_rows:,} satır")

    # 2. TRAFİK VERİSİ (7 gün)
    print("\n📶 2. Trafik verisi yükleniyor (7 gün)...")
    traffic_files = [
        "sms-call-internet-mi-2013-11-01.csv",
        "sms-call-internet-mi-2013-11-02.csv",
        "sms-call-internet-mi-2013-11-03.csv",
        "sms-call-internet-mi-2013-11-04.csv",
        "sms-call-internet-mi-2013-11-05.csv",
        "sms-call-internet-mi-2013-11-06.csv",
        "sms-call-internet-mi-2013-11-07.csv"
    ]

    for i, file in enumerate(traffic_files, 1):
        print(f"   📅 Gün {i}/7: {file}")
        file_path = BASE_PATH + file
        date_str = file[-15:-4]  # "2013-11-01" formatı

        # CSV'yi oku
        df_chunk = pd.read_csv(file_path)

        # datetime sütununu dönüştür
        df_chunk['datetime'] = pd.to_datetime(df_chunk['datetime'])

        # load_date ekle
        df_chunk['load_date'] = pd.to_datetime(date_str).date()

        # Tablo adı: traffic_YYYYMMDD
        table_name = f"traffic_{date_str.replace('-', '')}"
        table_id = f"{PROJECT_ID}.{DATASET_ID}.{table_name}"

        job = client.load_table_from_dataframe(df_chunk, table_id, job_config=job_config)
        job.result()
        print(f"     ✅ {table_name}: {df_chunk.shape[0]:,} satır")

    # 3. HAREKET VERİSİ (7 gün)
    print("\n🗺️  3. Hareket verisi yükleniyor (7 gün)...")
    movement_files = [
        "mi-to-provinces-2013-11-01.csv",
        "mi-to-provinces-2013-11-02.csv",
        "mi-to-provinces-2013-11-03.csv",
        "mi-to-provinces-2013-11-04.csv",
        "mi-to-provinces-2013-11-05.csv",
        "mi-to-provinces-2013-11-06.csv",
        "mi-to-provinces-2013-11-07.csv"
    ]

    for i, file in enumerate(movement_files, 1):
        print(f"   📅 Gün {i}/7: {file}")
        file_path = BASE_PATH + file
        date_str = file[-15:-4]

        df_chunk = pd.read_csv(file_path)
        df_chunk['datetime'] = pd.to_datetime(df_chunk['datetime'])
        df_chunk['load_date'] = pd.to_datetime(date_str).date()

        table_name = f"movement_{date_str.replace('-', '')}"
        table_id = f"{PROJECT_ID}.{DATASET_ID}.{table_name}"

        job = client.load_table_from_dataframe(df_chunk, table_id, job_config=job_config)
        job.result()
        print(f"     ✅ {table_name}: {df_chunk.shape[0]:,} satır")

    # 4. GEOJSON DOSYALARI (DÜZELTİLMİŞ)
    print("\n🗺️  4. Coğrafi veriler yükleniyor...")

    # Milano grid - DOĞRU SÜTUN İSİMLERİYLE
    print("   📍 Milano grid...")
    grid_path = BASE_PATH + "milano-grid.geojson"
    grid_gdf = gpd.read_file(grid_path)

    # 'cellId' sütununu 'CellID' olarak standardize et
    grid_gdf = grid_gdf.rename(columns={'cellId': 'CellID'})

    # GeoJSON'dan DataFrame'e - sadece gerekli sütunlar
    grid_df = pd.DataFrame({
        'CellID': grid_gdf['CellID'],
        'geometry': grid_gdf.geometry.astype(str)
    })

    table_id = f"{PROJECT_ID}.{DATASET_ID}.grid_locations"
    job = client.load_table_from_dataframe(grid_df, table_id, job_config=job_config)
    job.result()
    print(f"     ✅ grid_locations: {grid_df.shape[0]:,} satır")
    print(f"     🔍 Örnek CellID'ler: {grid_df['CellID'].head().tolist()}")

    # İtalya eyaletleri - DOĞRU SÜTUN İSİMLERİYLE
    print("   🇮🇹 İtalya eyaletleri...")
    provinces_path = BASE_PATH + "Italian_provinces.geojson"
    provinces_gdf = gpd.read_file(provinces_path)

    # 'PROVINCIA' sütununu 'province_name' olarak standardize et
    provinces_df = pd.DataFrame({
        'province_name': provinces_gdf['PROVINCIA'],
        'province_code': provinces_gdf['SIGLA'],  # SIGLA = kısaltma (TO, MI vb.)
        'shape_area': provinces_gdf['SHAPE_AREA'],
        'geometry': provinces_gdf.geometry.astype(str)
    })

    table_id = f"{PROJECT_ID}.{DATASET_ID}.province_boundaries"
    job = client.load_table_from_dataframe(provinces_df, table_id, job_config=job_config)
    job.result()
    print(f"     ✅ province_boundaries: {provinces_df.shape[0]:,} satır")
    print(f"     🔍 Örnek eyaletler: {provinces_df['province_name'].head().tolist()}")

    print("\n🎉 TÜM TABLOLAR BAŞARIYLA YÜKLENDİ!")

# Fonksiyonu çalıştır
load_all_tables()

## Birleşik Tablolar Oluşturma

### *7 günlük trafik ve hareket verisi için birleşik view'ler*

In [ ]:
def create_combined_views():
    """Birleşik view'lar oluştur"""

    # 1. TÜM TRAFİK VERİSİ (7 gün)
    print("🔄 Tüm trafik view'ı oluşturuluyor...")

    query = f"""
    CREATE OR REPLACE VIEW `{PROJECT_ID}.{DATASET_ID}.traffic_all_days` AS
    SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.traffic_20131101` UNION ALL
    SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.traffic_20131102` UNION ALL
    SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.traffic_20131103` UNION ALL
    SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.traffic_20131104` UNION ALL
    SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.traffic_20131105` UNION ALL
    SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.traffic_20131106` UNION ALL
    SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.traffic_20131107`
    """

    client.query(query).result()
    print("   ✅ traffic_all_days view oluşturuldu")

    # 2. TÜM HAREKET VERİSİ (7 gün)
    print("🔄 Tüm hareket view'ı oluşturuluyor...")

    query = f"""
    CREATE OR REPLACE VIEW `{PROJECT_ID}.{DATASET_ID}.movement_all_days` AS
    SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.movement_20131101` UNION ALL
    SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.movement_20131102` UNION ALL
    SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.movement_20131103` UNION ALL
    SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.movement_20131104` UNION ALL
    SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.movement_20131105` UNION ALL
    SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.movement_20131106` UNION ALL
    SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.movement_20131107`
    """

    client.query(query).result()
    print("   ✅ movement_all_days view oluşturuldu")

    # 3. GÜNLÜK ÖZET TABLOSU
    print("🔄 Günlük özet view'ı oluşturuluyor...")

    query = f"""
    CREATE OR REPLACE VIEW `{PROJECT_ID}.{DATASET_ID}.daily_summary` AS
    SELECT
        DATE(datetime) as date,
        EXTRACT(HOUR FROM datetime) as hour,
        COUNT(DISTINCT CellID) as unique_cells,
        SUM(COALESCE(smsin, 0)) as total_smsin,
        SUM(COALESCE(smsout, 0)) as total_smsout,
        SUM(COALESCE(callin, 0)) as total_callin,
        SUM(COALESCE(callout, 0)) as total_callout,
        SUM(COALESCE(internet, 0)) as total_internet
    FROM `{PROJECT_ID}.{DATASET_ID}.traffic_all_days`
    GROUP BY date, hour
    ORDER BY date, hour
    """

    client.query(query).result()
    print("   ✅ daily_summary view oluşturuldu")

    # 4. EYALET BAZLI HAREKET ÖZETİ
    print("🔄 Eyalet hareket özeti view'ı oluşturuluyor...")

    query = f"""
    CREATE OR REPLACE VIEW `{PROJECT_ID}.{DATASET_ID}.province_movement_summary` AS
    SELECT
        DATE(datetime) as date,
        provinceName,
        SUM(COALESCE(cell2Province, 0)) as total_from_milano,
        SUM(COALESCE(Province2cell, 0)) as total_to_milano,
        COUNT(DISTINCT CellID) as unique_cells_connected
    FROM `{PROJECT_ID}.{DATASET_ID}.movement_all_days`
    GROUP BY date, provinceName
    ORDER BY date, total_from_milano DESC
    """

    client.query(query).result()
    print("   ✅ province_movement_summary view oluşturuldu")

    print("\n🎉 TÜM VIEW'LAR OLUŞTURULDU!")

# View'ları oluştur
create_combined_views()

## Doğrulama Sorguları

In [ ]:
def verify_tables():
    """Tabloları doğrula"""

    print("🔍 TABLO DOĞRULAMASI")
    print("=" * 50)

    # Dataset referansı
    dataset_ref = client.dataset(DATASET_ID)

    table_info = []
    try:
        # Dataset'teki tüm tabloları listele
        tables = list(client.list_tables(dataset_ref))

        for table in tables:
            # Tablo referansı
            table_ref = dataset_ref.table(table.table_id)
            table_obj = client.get_table(table_ref)

            table_info.append({
                'Table': table.table_id,
                'Rows': f"{table_obj.num_rows:,}",
                'Size (MB)': f"{table_obj.num_bytes / (1024*1024):.1f}",
                'Type': table_obj.table_type
            })

        # DataFrame olarak göster
        df_info = pd.DataFrame(table_info)
        print(df_info.to_string(index=False))

        # Toplam istatistik
        total_rows = sum([table_obj.num_rows for table in tables])
        total_size_mb = sum([table_obj.num_bytes for table in tables]) / (1024*1024)

        print(f"\n📈 TOPLAM: {len(tables)} tablo, {total_rows:,} satır, {total_size_mb:.1f} MB")

    except Exception as e:
        print(f"Hata: {e}")
        return None

    return df_info

# Doğrulama çalıştır
print("\n" + "="*50)
print("TABLO DOĞRULAMA ÇALIŞTIRILIYOR...")
print("="*50)
df_tables = verify_tables()

## **RELEASE THE KRAKEN**

In [ ]:
# Tüm işlemi başlat
print("🚀 BIGQUERY VERİ YÜKLEME BAŞLIYOR...")
print(f"📂 Project: {PROJECT_ID}")
print(f"📂 Dataset: {DATASET_ID}")
print(f"📁 Path: {BASE_PATH}")
print("=" * 50)

try:
    # 1. Tüm tabloları yükle
    print("\n📦 1. TABLO YÜKLEME İŞLEMİ")
    print("-" * 30)
    load_all_tables()

    print("\n✅ TABLO YÜKLEME TAMAMLANDI")
    print("=" * 50)

    # 2. View'ları oluştur
    print("\n🔗 2. VIEW OLUŞTURMA İŞLEMİ")
    print("-" * 30)
    create_combined_views()

    print("\n✅ VIEW OLUŞTURMA TAMAMLANDI")
    print("=" * 50)

    # 3. Doğrulama
    print("\n🔍 3. DOĞRULAMA İŞLEMİ")
    print("-" * 30)
    df_tables = verify_tables()

    print("\n" + "="*50)
    print("🎉 PROJE HAZIR! NATURE ANALİZLERİNE BAŞLAYABİLİRSİNİZ.")
    print("="*50)

    # Ek bilgiler
    print("\n📊 HAZIR TABLOLAR:")
    print("   1. traffic_all_days - Tüm trafik verisi")
    print("   2. movement_all_days - Tüm hareket verisi")
    print("   3. daily_summary - Günlük trafik özeti")
    print("   4. province_movement_summary - Eyalet hareket özeti")
    print("   5. census_data - Demografik veri")
    print("   6. grid_locations - Milano grid hücreleri")
    print("   7. province_boundaries - İtalya eyalet sınırları")

except Exception as e:
    print(f"\n❌ HATA OLUŞTU: {e}")
    print("\nHata detayı:")
    import traceback
    traceback.print_exc()

 *  *BigQuery'de 21 tablo oluşturuldu ve veri yüklendi*
 *  *4 birleşik view oluşturuldu (traffic_all_days, movement_all_days,daily_summary, province_movement_summary)*
 *  *Nature metodolojisine tam uyumlu veri yapısı kuruldu*
 *  *Gerçek veri seti olduğu keşfedildi (sentetik değil, Telefonica Milano 2013 verisi)*

# **Summarise : Gün 1 (02.12.2025 Salı): PROJE HAZIRLIĞI VE VERİYİ ANLAMA - TAMAMLANDI**

* ✓ Makale ve veri seti incelemesi tamamlanacak
* ✓ Proje hedefleri netleştirilecek
* ✓ Çalışma ortamı kurulacak